In [1]:
import tensorflow as tf
import numpy as np
import os
import pathlib
from deepsudoku import load_supervised_data, metrics_and_callbacks, REPO_PATH

os.chdir(REPO_PATH)

BATCHSIZE = 256 * 4

2023-07-26 18:15:39.245439: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 18:15:40.626805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
inputs, labels = load_supervised_data.get_data("train")[:2]
index = np.arange(len(inputs))
np.random.shuffle(index)
inputs, labels = inputs[index], labels[index]

train_ds = load_supervised_data.get_tf_dataset(inputs, labels)
train_ds = train_ds.batch(BATCHSIZE).shuffle(100)

validation_data_by_difficulty = load_supervised_data.get_data_by_difficulty("val")

2023-07-26 18:15:45.768558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-26 18:15:46.064892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-26 18:15:46.065575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
model_inputs = tf.keras.Input((9,9,10))
x = model_inputs

x = tf.keras.layers.Conv2D(32,3, padding= "same", activation= "relu")(x)
x = tf.keras.layers.Conv2D(64,3, padding= "same", activation= "relu")(x)
x = tf.keras.layers.Conv2D(128,3, padding= "same", activation= "relu")(x)
x = tf.keras.layers.Conv2D(256,9, padding= "same", activation= "relu")(x)
x = tf.keras.layers.Conv2D(9,9, padding= "same", activation= "softmax")(x)

model = tf.keras.Model(model_inputs,x)

In [4]:
start_slices = []
size = [-1, 3, 3, -1]

for i in range(0, 9, 3):
    for y in range(0, 9, 3):
        start_slices.append([0,i,y,0])

start_slices = tf.convert_to_tensor(start_slices)
size = tf.convert_to_tensor(size)

MSE = tf.keras.losses.MeanSquaredError()
CROSS = tf.keras.losses.SparseCategoricalCrossentropy()
# LAMBDA = 0.5

# @tf.function
# def loss(y_true, y_pred):
    
#     reg_term = tf.convert_to_tensor(0, tf.float32)
    
#     # blocks
    
#     for start in start_slices:
    
#         reg = tf.slice(y_pred, start,size)
#         reg = tf.reduce_sum(reg, axis = [1, 2])
#         reg_term += MSE(reg, tf.ones_like(reg))
    
#     reg = tf.reduce_sum(y_pred, axis = 1)
#     reg_term += MSE(reg, tf.ones_like(reg))
    
#     reg = tf.reduce_sum(y_pred, axis = 2)
#     reg_term += MSE(reg, tf.ones_like(reg))
    
#     return CROSS(y_true,y_pred) + LAMBDA * reg_term

model.compile("Adam", CROSS, metrics = ["accuracy", metrics_and_callbacks.SudokuWinRate()])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 10)]        0         
                                                                 
 conv2d (Conv2D)             (None, 9, 9, 32)          2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 9, 9, 256)         2654464   
                                                                 
 conv2d_4 (Conv2D)           (None, 9, 9, 9)           186633    
                                                                 
Total params: 2,936,361
Trainable params: 2,936,361
Non-train

In [5]:
callbacks= [
    # tf.keras.callbacks.ReduceLROnPlateau("val_sudoku_win_rate/insane", mode = "max", patience = 5),
    # tf.keras.callbacks.EarlyStopping("val_sudoku_win_rate/insane", mode = "max", patience = 10),
    tf.keras.callbacks.ModelCheckpoint("ckpt/cp-{epoch:04d}.ckpt", save_weights_only = True),
    metrics_and_callbacks.SudokuWinRateByDifficulty(validation_data_by_difficulty, BATCHSIZE),
    tf.keras.callbacks.CSVLogger("logs.csv"),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [12]:
model = tf.keras.models.load_model(
    "ckpt/model_epoch_71.ckpt",
    custom_objects={"SudokuWinRate": metrics_and_callbacks.SudokuWinRate}
)

In [13]:
hist = model.fit(train_ds, epochs = 100, callbacks = callbacks, initial_epoch = 71)

Epoch 72/100


2023-07-26 18:19:28.677510: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [1460000,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 18:19:28.677815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [1460000,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 18:19:32.234368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-26 18:19:34.182732: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 18:19:34.183243: I tensorflow/

1426/1426 [==============================] - ETA: 0s - loss: 1.5912e-04 - accuracy: 1.0000 - sudoku_win_rate: 0.9987

2023-07-26 18:23:50.520246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [25000,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 18:23:52.913243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [53500,9,9]
	 [[{{node Placeholder/_1}}]]
2023-07-26 18:23:56.664445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shap

1426/1426 [==============================] - 276s 184ms/step - loss: 1.5912e-04 - accuracy: 1.0000 - sudoku_win_rate: 0.9987 - val_sudoku_win_rate/easy: 1.0000 - val_sudoku_win_rate/medium: 0.9998 - val_sudoku_win_rate/hard: 0.9987 - val_sudoku_win_rate/extreme: 0.9939 - val_sudoku_win_rate/insane: 0.9859
Epoch 73/100
1426/1426 [==============================] - 257s 180ms/step - loss: 1.5328e-04 - accuracy: 1.0000 - sudoku_win_rate: 0.9987 - val_sudoku_win_rate/easy: 1.0000 - val_sudoku_win_rate/medium: 0.9999 - val_sudoku_win_rate/hard: 0.9990 - val_sudoku_win_rate/extreme: 0.9949 - val_sudoku_win_rate/insane: 0.9872
Epoch 74/100
1426/1426 [==============================] - 253s 177ms/step - loss: 1.5386e-04 - accuracy: 1.0000 - sudoku_win_rate: 0.9987 - val_sudoku_win_rate/easy: 1.0000 - val_sudoku_win_rate/medium: 0.9998 - val_sudoku_win_rate/hard: 0.9990 - val_sudoku_win_rate/extreme: 0.9942 - val_sudoku_win_rate/insane: 0.9875
Epoch 75/100
1426/1426 [=============================

KeyboardInterrupt: 

In [14]:
model.save("ckpt/final_model.ckpt")

INFO:tensorflow:Assets written to: ckpt/final_model.ckpt/assets


INFO:tensorflow:Assets written to: ckpt/final_model.ckpt/assets
